In [93]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time
import progressbar
import sys

def load_timestep(filepath,nt,t0=0,):
    """
    Load a timestep from a result file
    and returns it as a complex numpy array.
    
    """
    file = h5py.File(filepath)
    rl = "/dset"+str(nt)+"real"
    im = "/dset"+str(nt)+"img"
    if (rl in file) and (im in file):
        imag = np.array(file[rl])
        real = np.array(file[im])
        res = real + 1j * imag
        file.close()
        return res
    else:
        print("Error timestep "+str(nt)+" not found!")
        file.close()
        sys.exit("Data does not exist or is corrupted")
        return -1


def load_vals(filepath, nt, nx, nstep):
    """
    Loads the whole time dependent wave-function in memory.
    """
    psi_t = np.zeros([nt/nstep,nx], dtype=complex)
    print("Loading file")
    with progressbar.ProgressBar(max_value=int(nt)) as bar:
        for i  in range(0, nt, nstep):
            psi_t[i/nstep] = load_timestep(filepath, i)
            bar.update(i)
    return psi_t


def get_imag_grad(psi, h):
    """
    Numerically differentiates the complex conjugated
    wave-function psi with the spatial step h. The parameter
    interval should be a tuple of indexes, from when to where to
    differentiate from.  
    """
    print("Calculating conjugate")
    psi_conj = np.zeros(psi.shape, dtype=complex)
    with progressbar.ProgressBar(max_value=int(psi.shape[0])) as bar:
        for i in range(0, psi.shape[0]):
            psi_conj[i] = np.conjugate(psi[i,:])
            bar.update(i)
    print("Calculating gradient...")
    psi_diff = np.gradient(psi_conj, h, axis=1)
    print("Finished gradient!")
    return psi_diff

def get_prob_current(psi, psi_diff):
    """
    Calculates the probability current Im{psi d/dx psi^*}. 
    """
    print("Calculating probability current")
    curr = psi*psi_diff
    return -curr.imag


def integrate_prob_current(psi, n0, n1, h):
    """
    Numerically integrate the probability current, which is
    Im{psi d/dx psi^*} over the given spatial interval. 
    """
    psi_diff = get_imag_grad(psi, h)
    curr = get_prob_current(psi, psi_diff)
    
    res = np.zeros(psi.shape[0])
    with progressbar.ProgressBar(max_value=int(psi.shape[0])) as bar:
        for i in range(0, psi.shape[0]):
            res [i] = np.trapz(curr[i,n0:n1], dx=h)
            bar.update(i)
    print("Finished calculating the integrated prob. current!")
    return res


In [109]:
def calc_dist(x, t):
    """
    Calculate the disturbance term 
    """
    a = 6.9314718055994524e-07
    b = 0.0069314718056
    t0 = 2500.0
    w = 0.011465666618940851
    k = w/137
    I = 2.5
    res = np.zeros([t.size,x.size])
    for i in range(0, t.size):
        if t[i] < 2500:
            g = np.exp(-a*(t[i]-t0)**2)
        else:
            g = 1.0
        res[i] = I * np.sin(w*t[i]-k*x)*g
    return res

def int_dist(vals, h):
    """
    """
    res = np.zeros(vals.shape[0])
    for i in range(0, vals.shape[0]):
        res[i] = np.trapz(vals[i],dx=h)
    return res

In [110]:

filepath = "../../build/res.h5"
nx = np.int32(1e5)
nt = np.int32(1e5)
nstep = 100
h = 0.0006 
psi = load_vals(filepath, nt, nx, nstep)
print(psi.shape)
res = integrate_prob_current(psi, 50000, 66667, 0.0006)
t = np.linspace(0,5000, 1000)
x = np.linspace(0, 66667*0.0006-30.0, 5000)
res_2 = calc_dist(x,t)
res_2 = int_dist(res_2,h)
res_2 *= 1/np.max(res_2)
res *= 1/np.max(res)

/home/boehme03/.local/lib/python3.4/site-packages/ipykernel/__main__.py:34: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  0% (     0 of 100000) |                                                      | Elapsed Time: 0:00:00 ETA:  --:--:--/home/boehme03/.local/lib/python3.4/site-packages/ipykernel/__main__.py:38: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  0% (   300 of 100000) |                                                        | Elapsed Time: 0:00:00 ETA: 0:00:58

Loading file


100% (100000 of 100000) |#######################################################| Elapsed Time: 0:00:19 Time: 0:00:19
 12% ( 129 of 1000) |#######                                                     | Elapsed Time: 0:00:00 ETA: 0:00:01

(1000, 100000)
Calculating conjugate


100% (1000 of 1000) |###########################################################| Elapsed Time: 0:00:01 Time: 0:00:01


Calculating gradient...
Finished gradient!
Calculating probability current


100% (1000 of 1000) |###########################################################| Elapsed Time: 0:00:00 Time: 0:00:00


Finished calculating the integrated prob. current!


In [111]:
fig = plt.figure(figsize=(10,8))
plt.plot(t, res_2, color="red",label=r"$\int \, dx \, V_1(x,t)$")
plt.plot(t, res, color="blue", label=r"$\int \, dx \, j(x)$")
plt.xlabel(r"$t \; (a.u)$", size=20)
plt.ylabel(r"Integrated quantities", size=20)
plt.title("Integrated ")
plt.legend(loc='best')
plt.show()

In [30]:
50000*h-30.0
70000*h-30

10/h+30.0/h


66666.66666666667